In [0]:
%pip install langchain_openai
%pip install langgraph
%pip install langchain_core
%pip install databricks_langchain

In [0]:
dbutils.library.restartPython()

In [0]:
from typing import TypedDict, List, Optional
from langgraph.graph import StateGraph, END, START
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
# from langchain_openai import ChatOpenAI
from databricks_langchain import ChatDatabricks
from IPython.display import Image, display
from langgraph.checkpoint.memory import MemorySaver

In [0]:
DATABRICKS_HOST = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiUrl().get()
DATABRICKS_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

print(f"workspace_url: {DATABRICKS_HOST}, token: {DATABRICKS_TOKEN}")

In [0]:
model_name = "databricks-gpt-5-nano"
base_url = f"https://{DATABRICKS_HOST}/serving-endpoints/{model_name}/invocations"

In [0]:
llm = ChatDatabricks(model="databricks-gpt-5-nano")

checkpointer = MemorySaver()

In [0]:
# ---- Agent State ----
class AgentState(TypedDict):
    pirate_messages: List[BaseMessage]
    pirate_input: Optional[str]

    translator_messages: List[BaseMessage]
    translator_input: Optional[str]

In [0]:
system_prompt_msg_pirate = SystemMessage(content="You are a helpful assistant that speaks like a pirate, but likes to be sarcastic, and tries to question everything, but wants to the right thing")

def chat_pirate_agent(state: AgentState):
    messages = state.get("pirate_messages", [])

    if state.get("pirate_input"):
        messages = messages + [HumanMessage(content=state["pirate_input"])]

    response = llm.invoke([system_prompt_msg_pirate] + messages)

    return {
        "pirate_messages": messages + [response],
        "pirate_input": None,  # clear input after use
    }

In [0]:
system_prompt_msg_translator = SystemMessage(content="You are a helpful translator that accepts weird and wonderful ways of speaking, including non-formal language, and translate that into something that is more understandable for a business audience. Don't ask any additional information, just provide the translation best you can")

def chat_translator_agent(state: AgentState):
    messages = state.get("translator_messages", [])

    if state.get("translator_input"):
        messages = messages + [HumanMessage(content=state["translator_input"])]

    response = llm.invoke([system_prompt_msg_translator] + messages)

    return {
        "translator_messages": messages + [response],
        "translator_input": None,  # clear input after use
    }

In [0]:
# ---- Build Graph ----
graph = StateGraph(AgentState)

graph.add_node("pirate_agent",chat_pirate_agent)
graph.add_node("translator_agent",chat_translator_agent)

graph.add_edge(START, "pirate_agent")
graph.add_edge("pirate_agent","translator_agent")
graph.add_edge("translator_agent",END)

compiled_graph = graph.compile(checkpointer=checkpointer)
display(Image(compiled_graph.get_graph().draw_mermaid_png()))

In [0]:
def chat(msg: str):
    response = compiled_graph.invoke(
        {
            "input_input": msg
        },
        config={"configurable": {"thread_id": "session-1"}}
        )
    return response["translator_messages"][-1].content

In [0]:
chat("whats up")

In [0]:
chat("my name is dennis, remember this")

In [0]:
chat("whats my name")

In [0]:
chat("ask me some stuff to get to know me")